### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [3]:
reviews.head() #movie_id and rating of it (I want the highest rating for each movie)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
movie_ratings = reviews.groupby('movie_id')['rating']
print(movie_ratings.mean()[:10]) #gives AVG rating for eaach movie
print(movie_ratings.count()[:10]) #how many times that particular movie was rated

movie_id
8       5.000000
10     10.000000
12     10.000000
25      8.000000
91      6.000000
417     8.368421
439     6.750000
443     8.000000
628     4.500000
833     3.000000
Name: rating, dtype: float64
movie_id
8       1
10      1
12      1
25      1
91      1
417    19
439     4
443     1
628     2
833     1
Name: rating, dtype: int64


In [23]:
reviews.groupby('movie_id').max() #max: newest movie, min: oldest movie

,user_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
movie_id,,,,,,,,,,,,,,,,,,,,,
8,40425,5,1396981211,2014-04-08 18:20:11,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10,48654,10,1412878553,2014-10-09 18:15:53,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
12,38857,10,1439248579,2015-08-10 23:16:19,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
25,30852,8,1488189899,2017-02-27 10:04:59,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
91,9720,6,1385233195,2013-11-23 18:59:55,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
417,52959,10,1530047326,2018-06-26 21:08:46,0,0,0,0,0,0,...,0,1,0,1,0,1,1,1,0,1
439,47839,8,1529809149,2018-06-24 02:59:09,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,1
443,20392,8,1477256912,2016-10-23 21:08:32,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
628,38951,5,1385729728,2013-11-29 12:55:28,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,0,0


In [31]:
movies.set_index('movie_id').head(n=2)

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
movie_id,,,,,,,,,,,,,,,,,,,,,
8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [32]:
#helper function
def create_ranked_df(movies, reviews):
    movie_ratings = reviews.groupby('movie_id')['rating']
    avg_ratings = movie_ratings.mean()
    num_ratings = movie_ratings.count()
    
    last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date']) #max: newest movie
    last_rating.columns = ['last_rating']
    
    rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings})
    rating_count_df = rating_count_df.join(last_rating)
    
    # merge with the movies dataset
    movie_recs = movies.set_index('movie_id').join(rating_count_df)

    # sort by top avg rating and number of ratings
    ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

    # for edge cases - subset the movie list to those with only 5 or more reviews
    ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]
        
    return ranked_movies


def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    
    top_movies = list(ranked_movies['movie'][:n_top])

    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [33]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1
ranked_movies = create_ranked_df(movies, reviews) # only run this once - it is not fast

recs_20_for_1 = popular_recommendations('1', 20, ranked_movies)

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5, ranked_movies)

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100, ranked_movies)

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35, ranked_movies)



In [38]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


In [39]:
ranked_movies.head()

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.0,28,2016-01-08 00:44:43
5688932,Sorry to Bother You (2018),Comedy|Fantasy|Sci-Fi,2018,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.0,14,2018-06-17 01:44:48
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-10 22:56:01
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,10.0,6,2016-01-23 00:30:44


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [40]:
ranked_movies[ranked_movies['date'].isin(['2015'])]

,movie,genre,date,1800's,1900's,2000's,History,News,Horror,Musical,...,Action,Documentary,Animation,Comedy,Short,Western,Thriller,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,10.000000,48,2016-08-14 17:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,28,2016-01-08 00:44:43
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,10.000000,5,2017-09-09 13:51:48
5733210,Terra (2015),Documentary,2015,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,9.333333,6,2017-03-22 11:13:33
4973112,Hitting the Apex (2015),Documentary,2015,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,9.166667,6,2017-10-25 18:50:08
5027202,Colombia magia salvaje (2015),Documentary,2015,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,9.142857,7,2017-06-19 23:17:39
5098836,I Believe in Miracles (2015),Documentary|History|Sport,2015,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,9.100000,10,2018-04-27 22:13:39
3863552,Bajrangi Bhaijaan (2015),Comedy|Drama,2015,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,9.026042,192,2018-04-22 13:43:41
3270538,Requiem for the American Dream (2015),Documentary|Biography|News,2015,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,8.950000,20,2018-04-23 12:25:24


In [44]:
ranked_movies[['History','Sci-Fi']].sum(axis=1)

movie_id
4921860    0
5262972    0
5688932    1
2737018    0
2560840    0
2219210    1
4448444    0
5131914    1
2059318    1
1431149    0
5512872    0
4148400    1
6798422    0
111341     0
423176     0
1629443    0
2592910    0
58888      0
2396421    0
12364      0
57565      0
6054758    0
5134588    0
363473     0
2265179    0
5323386    0
29843      0
45274      0
2357788    0
6781498    0
          ..
2088923    0
3203620    0
3727824    0
2769184    1
466342     0
2006801    0
3283792    0
2325518    0
185183     1
1540767    0
3551400    0
2357489    0
110978     0
5988370    0
829176     0
3104304    1
2622826    1
3561236    1
3036740    0
1098327    0
2147365    0
105585     0
2573750    0
5669936    1
1213644    0
60666      0
3108604    0
3187378    0
4458206    0
7923374    0
Length: 9974, dtype: int64

In [49]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years, genres):
    
    # Filter movies based on year and genre
    ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    num_genre_match = ranked_movies[genres].sum(axis=1)
    ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies


In [57]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
recs_20_for_1_filtered = popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018'],genres=['Sci-Fi'])

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered('70000', 100, ranked_movies, years=['2015', '2016'],genres=['History', 'News'])

In [58]:
recs_5_for_53968_filtered

['I Believe in Miracles (2015)',
 'Requiem for the American Dream (2015)',
 'The Mask You Live In (2015)',
 'Sado (2015)',
 'Hatred (2016)',
 'Nise - O Coração da Loucura (2015)',
 'Straight Outta Compton (2015)',
 'Manjhi: The Mountain Man (2015)',
 'Only the Dead (2015)',
 'Spotlight (2015)',
 'Under sandet (2015)',
 'Airlift (2016)',
 'Best of Enemies (2015)',
 'All Things Must Pass: The Rise and Fall of Tower Records (2015)',
 'Colonia (2015)',
 'Daeho (2015)',
 'Eddie the Eagle (2016)',
 'Kainan 1890 (2015)',
 'Anthropoid (2016)',
 'Patriots Day (2016)',
 'Miekkailija (2015)',
 'Silence (2016)',
 'Suffragette (2015)',
 'The Founder (2016)',
 'Saul fia (2015)',
 'Woman in Gold (2015)',
 'Assassination (2015)',
 'The 33 (2015)',
 'Bajirao Mastani (2015)',
 'Denial (2016)',
 'A Tale of Love and Darkness (2015)',
 'The Finest Hours (2016)',
 'The Stanford Prison Experiment (2015)',
 'The Propaganda Game (2015)',
 'In-cheon sang-ryuk jak-jeon (2016)',
 'Banking on Bitcoin (2016)',
 'Dv